In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

deprecation 과 같은 경고메시지는 무시하도록 설정합니다.


In [2]:
# 경고 메시지 무시
import warnings

warnings.filterwarnings("ignore")

## VectorStoreRetriever


In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma


# 텍스트를 600자 단위로 분할
text_splitter = CharacterTextSplitter(chunk_size=600, chunk_overlap=0)

# TextLoader 를 통해 텍스트 파일을 로드
split_docs = TextLoader("data/appendix-keywords.txt").load_and_split(text_splitter)

# Chroma 를 통해 벡터 저장소 생성
chroma_db = Chroma.from_documents(split_docs, OpenAIEmbeddings())

# retriever 생성
retriever = chroma_db.as_retriever()

# similarity_search 를 통해 유사도 높은 1개 문서를 검색
relevant_docs = retriever.get_relevant_documents("Page Rank 에 대하여 알려줘")

print(f"문서의 개수: {len(relevant_docs)}")
print("[검색 결과]\n")
print(relevant_docs[0].page_content)

/home/samuel/Dev/langchain-kr/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


문서의 개수: 4
[검색 결과]

정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank

정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.
예시: 구글 검색 엔진은 페이지 랭크 알고리즘을 사용하여 검색 결과의 순위를 정합니다.
연관키워드: 검색 엔진 최적화, 웹 분석, 링크 분석

데이터 마이닝

정의: 데이터 마이닝은 대량의 데이터에서 유용한 정보를 발굴하는 과정입니다. 이는 통계, 머신러닝, 패턴 인식 등의 기술을 활용합니다.
예시: 소매업체가 고객 구매 데이터를 분석하여 판매 전략을 수립하는 것은 데이터 마이닝의 예입니다.
연관키워드: 빅데이터, 패턴 인식, 예측 분석

멀티모달 (Multimodal)


## MultiQueryRetriever

거리 기반 벡터 데이터베이스 검색은 쿼리를 고차원 공간에 임베드(표현)하고 '거리'를 기준으로 유사한 임베드 문서를 찾습니다. 그러나 쿼리 문구가 미묘하게 변경되거나 임베딩이 데이터의 의미를 제대로 포착하지 못하는 경우 검색 결과가 달라질 수 있습니다. 이러한 문제를 수동으로 해결하기 위해 즉각적인 엔지니어링/튜닝을 수행하기도 하지만, 이는 번거로운 작업일 수 있습니다.

**MultiQueryRetriever** 는 LLM을 사용해 주어진 사용자 입력 쿼리에 대해 서로 다른 관점에서 여러 쿼리를 생성함으로써 프롬프트 튜닝 프로세스를 자동화합니다. 각 쿼리에 대해 관련 문서 집합을 검색하고 모든 쿼리에서 고유한 유니온을 사용하여 잠재적으로 관련성이 높은 더 큰 문서 집합을 가져옵니다. 동일한 질문에 대해 여러 관점을 생성함으로써, 멀티쿼리 리트리버는 거리 기반 검색의 일부 한계를 극복하고 더 풍부한 결과 세트를 얻을 수 있습니다.


In [4]:
# Build a sample vectorDB
from langchain.text_splitter import (
    CharacterTextSplitter,
)
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Load blog post
loader = TextLoader("data/appendix-keywords.txt")
data = loader.load()

# Split
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

# VectorDB
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

In [5]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

question = "Page Rank 에 대하여 알려줘"

llm = ChatOpenAI(temperature=0, model="gpt-4-turbo-preview")

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [6]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [7]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 페이지 랭크 알고리즘의 작동 원리에 대해 설명해줄 수 있나요?', '2. Page Rank 기술이 웹 페이지 순위 결정에 어떻게 사용되는지 알려주세요.', '3. 구글의 페이지 랭크 시스템에 대한 상세한 정보를 제공해줄 수 있나요?']


4

In [8]:
for i in range(len(unique_docs)):
    print(f"문서 {i+1}:\n{unique_docs[i].page_content}\n")
    print("===" * 20)

문서 1:
정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.
예시: 구글 검색 엔진은 페이지 랭크 알고리즘을 사용하여 검색 결과의 순위를 정합니다.
연관키워드: 검색 엔진 최적화, 웹 분석, 링크 분석

데이터 마이닝

정의: 데이터 마이닝은 대량의 데이터에서 유용한 정보를 발굴하는 과정입니다. 이는 통계, 머신러닝, 패턴 인식 등의 기술을 활용합니다.
예시: 소매업체가 고객 구매 데이터를 분석하여 판매 전략을 수립하는 것은 데이터 마이닝의 예입니다.
연관키워드: 빅데이터, 패턴 인식, 예측 분석

멀티모달 (Multimodal)

문서 2:
정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank

정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.
예시: 구글 검색 엔진은 페이지 랭크 알고리즘을 사용하여 검색 결과의 순위를 정합니다.
연관키워드: 검색 엔진 최적화, 웹 분석, 링크 분석

데이터 마이닝

정의: 데이터 마이닝은 대량의 데이터에서 유용한 정보를 발굴하는 과정입니다. 이는 통계, 머신러닝, 패턴 인식 등의 기술을 활용합니다.
예시: 소매업체가 고객 구매 데이터를 분석하여 판매 전략을 수립하는 것은 데이터 마이닝의 예입니다.
연관키워드: 빅데이터, 패턴 인식, 예측 분석

멀티모달 (Multimodal)

문서 3:
정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자

In [9]:
from typing import List

from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field


# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    # parse() 함수를 재정의하여, LLM 결과를 파싱
    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines.
    Original question: {question}""",
)
llm = ChatOpenAI(temperature=0)

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

/home/samuel/Dev/langchain-kr/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [10]:
# Run
retriever = MultiQueryRetriever(
    retriever=vectordb.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
unique_docs = retriever.get_relevant_documents(query="LLM 에 관한 내용을 알려줘!")

OutputParserException: Failed to parse LineList from completion 1. Got: 1 validation error for LineList
  Input should be a valid dictionary or instance of LineList [type=model_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.7/v/model_type

In [11]:
unique_docs[0].page_content

'정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.\n예시: 구글 검색 엔진은 페이지 랭크 알고리즘을 사용하여 검색 결과의 순위를 정합니다.\n연관키워드: 검색 엔진 최적화, 웹 분석, 링크 분석\n\n데이터 마이닝\n\n정의: 데이터 마이닝은 대량의 데이터에서 유용한 정보를 발굴하는 과정입니다. 이는 통계, 머신러닝, 패턴 인식 등의 기술을 활용합니다.\n예시: 소매업체가 고객 구매 데이터를 분석하여 판매 전략을 수립하는 것은 데이터 마이닝의 예입니다.\n연관키워드: 빅데이터, 패턴 인식, 예측 분석\n\n멀티모달 (Multimodal)'

In [12]:
print(f"문서의 개수: {len(unique_docs)}")

for i in range(len(unique_docs)):
    print(f"문서 {i+1}:\n{unique_docs[i].page_content}\n")
    print("===" * 20)

문서의 개수: 4
문서 1:
정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.
예시: 구글 검색 엔진은 페이지 랭크 알고리즘을 사용하여 검색 결과의 순위를 정합니다.
연관키워드: 검색 엔진 최적화, 웹 분석, 링크 분석

데이터 마이닝

정의: 데이터 마이닝은 대량의 데이터에서 유용한 정보를 발굴하는 과정입니다. 이는 통계, 머신러닝, 패턴 인식 등의 기술을 활용합니다.
예시: 소매업체가 고객 구매 데이터를 분석하여 판매 전략을 수립하는 것은 데이터 마이닝의 예입니다.
연관키워드: 빅데이터, 패턴 인식, 예측 분석

멀티모달 (Multimodal)

문서 2:
정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank

정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.
예시: 구글 검색 엔진은 페이지 랭크 알고리즘을 사용하여 검색 결과의 순위를 정합니다.
연관키워드: 검색 엔진 최적화, 웹 분석, 링크 분석

데이터 마이닝

정의: 데이터 마이닝은 대량의 데이터에서 유용한 정보를 발굴하는 과정입니다. 이는 통계, 머신러닝, 패턴 인식 등의 기술을 활용합니다.
예시: 소매업체가 고객 구매 데이터를 분석하여 판매 전략을 수립하는 것은 데이터 마이닝의 예입니다.
연관키워드: 빅데이터, 패턴 인식, 예측 분석

멀티모달 (Multimodal)

문서 3:
정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나

## Contextual compression

검색의 한 가지 문제점은 일반적으로 시스템에 데이터를 수집할 때 문서 저장 시스템이 직면하게 될 **특정 쿼리를 알 수 없다는 것** 입니다. 즉, 쿼리와 가장 관련성이 높은 정보가 관련 없는 **텍스트가 많은 문서에 묻힐 수 있습니다**. 애플리케이션을 통해 전체 문서를 전달하면 더 많은 비용이 드는 LLM 호출과 더 낮은 응답으로 이어질 수 있습니다.

컨텍스트 압축은 이 문제를 해결하기 위한 것입니다. 검색된 문서를 있는 그대로 즉시 반환하는 대신 주어진 쿼리의 컨텍스트를 사용하여 압축하여 관련 정보만 반환하도록 하는 것입니다. "여기서 '압축'이란 개별 문서의 내용을 압축하는 것과 문서를 일괄적으로 필터링하는 것을 모두 의미합니다.

`Contextual Compression Retriever`를 사용하려면 다음이 필요합니다:

- Base Retriever
- 문서 압축기(Document Compressor)

`Contextual Compression Retriever`는 쿼리를 기본 리트리버로 전달하고, 초기 문서를 가져와서 문서 압축기를 통과시킵니다. 문서 압축기는 문서 목록을 가져와서 문서 내용을 줄이거나 문서를 모두 삭제하여 압축합니다.


In [13]:
# Helper function for printing docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" +
                d.page_content for i, d in enumerate(docs)]
        )
    )

In [14]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

documents = TextLoader("data/appendix-keywords.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
retriever = FAISS.from_documents(texts, OpenAIEmbeddings()).as_retriever()

docs = retriever.get_relevant_documents("Open Source 에 대한 내용을 알려줘")
pretty_print_docs(docs)

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [15]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "HuggingFace 에 대한 내용을 알려줘"
)
pretty_print_docs(compressed_docs)

OutputParserException: Failed to parse LineList from completion 1. Got: 1 validation error for LineList
  Input should be a valid dictionary or instance of LineList [type=model_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.7/v/model_type

### LLMChainFilter

**LLMChainFilter** 는 약간 더 간단하지만 더 강력한 압축기로, 문서 내용을 조작하지 않고도 처음에 검색된 문서 중 어떤 것을 필터링하고 어떤 것을 반환할지 결정하기 위해 LLM 체인을 사용합니다.


In [16]:
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "HuggingFace 에 대한 내용을 알려줘"
)
pretty_print_docs(compressed_docs)

OutputParserException: Failed to parse LineList from completion 1. Got: 1 validation error for LineList
  Input should be a valid dictionary or instance of LineList [type=model_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.7/v/model_type

### EmbeddingsFilter

검색된 각 문서에 대해 추가 LLM 호출을 하는 것은 비용이 많이 들고 느립니다.

임베딩 필터는 문서와 쿼리를 임베딩하고 쿼리와 충분히 유사한 임베딩이 있는 문서만 반환함으로써 더 저렴하고 빠른 옵션을 제공합니다.


In [17]:
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.8)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "HuggingFace 에 대한 내용을 알려줘"
)
pretty_print_docs(compressed_docs)

OutputParserException: Failed to parse LineList from completion 1. Got: 1 validation error for LineList
  Input should be a valid dictionary or instance of LineList [type=model_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.7/v/model_type

### 압축기와 문서 변환기를 함께 묶기

DocumentCompressorPipeline을 사용하면 여러 압축기를 순서대로 쉽게 결합할 수도 있습니다. 압축기와 함께 컨텍스트 압축을 수행하지 않고 단순히 문서 세트에 대해 일부 변환을 수행하는 BaseDocumentTransformers를 파이프라인에 추가할 수 있습니다. 예를 들어 텍스트 스플리터를 문서 변환기로 사용하여 문서를 더 작은 조각으로 분할할 수 있고, 임베딩 유사성을 기반으로 중복 문서를 필터링하는 데 임베딩 중복 필터를 사용할 수 있습니다.

아래에서는 먼저 문서를 더 작은 덩어리로 분할한 다음 중복 문서를 제거하고 쿼리와의 관련성을 기준으로 필터링하는 방식으로 압축기 파이프라인을 만듭니다.


In [ ]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_transformers import EmbeddingsRedundantFilter

splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0, separator=". ")
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, redundant_filter, relevant_filter]
)

In [ ]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "HuggingFace 에 대한 내용을 알려줘"
)
pretty_print_docs(compressed_docs)

## Ensemble Retriever

앙상블 리트리버는 리트리버 목록을 입력으로 받아 각 리트리버의 get_relevant_documents() 메서드 결과를 앙상블하고 상호 순위 융합 알고리즘에 따라 결과를 재순위화합니다.

서로 다른 알고리즘의 강점을 활용함으로써 앙상블 리트리버는 어떤 단일 알고리즘보다 더 나은 성능을 얻을 수 있습니다.

가장 일반적인 패턴은 BM25와 같은 스파스 리트리버와 임베딩 유사도 같은 밀도 리트리버를 결합하는 것인데, 두 알고리즘의 강점은 상호 보완적이기 때문입니다. 이를 "하이브리드 검색"이라고도 합니다. 스파스 리트리버는 키워드를 기반으로 관련 문서를 찾는 데 능숙하고, 밀도 리트리버는 의미적 유사성을 기반으로 관련 문서를 찾는 데 능숙합니다.


In [18]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [21]:
doc_list = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
]

# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts(doc_list)
bm25_retriever.k = 2

embedding = OpenAIEmbeddings()
faiss_vectorstore = FAISS.from_texts(doc_list, embedding)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [22]:
relevant_docs = ensemble_retriever.get_relevant_documents("apples")
print(f"문서의 개수: {len(relevant_docs)}\n")
pretty_print_docs(relevant_docs)

문서의 개수: 2

Document 1:

I like apples
----------------------------------------------------------------------------------------------------
Document 2:

Apples and oranges are fruits


## Long-Context Reorder

모델의 아키텍처에 관계없이 검색된 문서를 10개 이상 포함하면 상당한 성능 저하가 발생합니다. 요약하자면: 모델이 긴 컨텍스트 중간에 관련 정보에 액세스해야 하는 경우, 제공된 문서를 무시하는 경향이 있습니다.

- 논문: https://arxiv.org/abs/2307.03172

이 문제를 방지하려면 검색 후 문서를 다시 정렬하여 성능 저하를 방지할 수 있습니다.


In [ ]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_transformers import (
    LongContextReorder,
)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI

# Get embeddings.
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

texts = [
    "Basquetball is a great sport.",
    "Fly me to the moon is one of my favourite songs.",
    "The Celtics are my favourite team.",
    "This is a document about the Boston Celtics",
    "I simply love going to the movies",
    "The Boston Celtics won the game by 20 points",
    "This is just a random text.",
    "Elden Ring is one of the best games in the last 15 years.",
    "L. Kornet is one of the best Celtics players.",
    "Larry Bird was an iconic NBA player.",
]

# Create a retriever
retriever = Chroma.from_texts(texts, embedding=OpenAIEmbeddings()).as_retriever(
    search_kwargs={"k": 10}
)
query = "What can you tell me about the Celtics?"

# Get relevant documents ordered by relevance score
docs = retriever.get_relevant_documents(query)
docs

In [ ]:
# 문서를 재정렬합니다:
# 관련성이 낮은 문서는 목록의 가운데에 배치합니다.
# 시작/끝에 관련성 높은 요소를 배치합니다.
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# 4개의 관련 문서가 시작과 끝에 있는지 확인합니다.
reordered_docs

## MultiVector Retriever

문서당 여러 개의 벡터를 저장하는 것이 유용할 때가 많습니다. 이것이 유용한 사용 사례는 여러 가지가 있습니다. LangChain에는 이러한 유형의 설정을 쉽게 쿼리할 수 있는 기본 멀티벡터 리트리버가 있습니다. 많은 복잡성은 문서당 여러 개의 벡터를 생성하는 방법에 있습니다. 이 노트북에서는 이러한 벡터를 생성하고 멀티벡터 리트리버를 사용하는 몇 가지 일반적인 방법을 다룹니다.

문서당 여러 개의 벡터를 만드는 방법은 다음과 같습니다:

더 작은 청크: 문서를 더 작은 청크로 나누고, 그 청크들을 임베드합니다(ParentDocumentRetriever).
요약: 각 문서에 대한 요약을 만들어 문서와 함께(또는 문서 대신) 임베드합니다.
가상 질문: 각 문서가 답변하기에 적절한 가상 질문을 만들어 문서와 함께(또는 문서 대신) 임베드합니다.
이 방법을 사용하면 임베딩을 수동으로 추가하는 다른 방법도 사용할 수 있습니다. 이 방법은 문서 복구로 이어져야 하는 질문이나 쿼리를 명시적으로 추가할 수 있어 더 많은 제어가 가능하기 때문에 유용합니다.


In [ ]:
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [ ]:
from langchain.storage import InMemoryByteStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [ ]:
loaders = [
    TextLoader("data/appendix-keywords.txt"),
    TextLoader("data/reference.txt"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)
split_docs = text_splitter.split_documents(docs)

In [ ]:
print(f"분할된 문서의 개수: {len(split_docs)}")

In [ ]:
# The vectorstore to use to index the child chunks
import uuid

vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

doc_ids = [str(uuid.uuid4()) for _ in docs]

In [ ]:
doc_ids

In [ ]:
# The splitter to use to create smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=300)

In [ ]:
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

In [ ]:
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [ ]:
# Vectorstore alone retrieves the small chunkss
search_result = retriever.vectorstore.similarity_search("DataFrame")[0]
print(search_result.page_content)

## 상위 문서 리트리버

검색을 위해 문서를 분할할 때 종종 상충되는 욕구가 있습니다:

임베딩된 문서가 그 의미를 가장 정확하게 반영할 수 있도록 문서를 작게 만들고 싶을 수 있습니다. 너무 길면 임베딩이 의미를 잃을 수 있습니다.
각 청크의 컨텍스트가 유지될 수 있는 충분한 길이의 문서가 필요합니다.
부모 문서 리트리버는 작은 데이터 청크를 분할하여 저장함으로써 그 균형을 맞춥니다. 검색하는 동안 먼저 작은 청크를 가져온 다음 해당 청크의 상위 ID를 조회하여 더 큰 문서를 반환합니다.

'상위 문서'는 작은 청크의 출처가 되는 문서를 의미합니다. 이는 전체 원시 문서일 수도 있고 더 큰 청크일 수도 있습니다.


In [ ]:
from langchain.retrievers import ParentDocumentRetriever

In [ ]:
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [ ]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(".", glob="data/*.txt")
docs = loader.load()

print(f"문서의 수: {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========= [앞부분] 미리보기 =========\n")
print(docs[0].page_content[:500])

In [ ]:
# This text splitter is used to create the child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)
# The storage layer for the parent documents
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [ ]:
retriever.add_documents(docs, ids=None)

이제 벡터 저장소 검색 기능을 호출해 보겠습니다. 작은 청크를 저장하고 있으므로 작은 청크를 반환하는 것을 볼 수 있을 것입니다.


In [ ]:
sub_docs = vectorstore.similarity_search("DataFrame")
len(sub_docs)

In [ ]:
sub_docs[0].metadata

In [ ]:
print(sub_docs[0].page_content)

이제 전체 리트리버에서 검색해 보겠습니다. 작은 청크가 있는 문서를 반환하므로 큰 문서를 반환해야 합니다.


In [ ]:
relevant_doc = retriever.get_relevant_documents("DataFrame")

In [ ]:
relevant_doc[0].metadata

In [ ]:
relevant_doc[0].page_content